In [ ]:
from IPython.display import Image
from IPython.core.display import HTML

# Lesson 2: Introduction to spikes

## Lesson goals:

1. Discuss solutions to two challenges from Lesson 1:
 - Write a function to measure the "input resistance" of a neuron compartment
 - Organize recordings and superimpose results from multiple simulations
2. Introduction to sodium and potassium channels involved in generating action potentials (spikes)
3. Introduction to NMODL, the language used to specify ion channel mechanisms in NEURON
4. Inserting "Hodgkin-Huxley" ion channels into a NEURON model compartment
5. Plotting the voltage-dependence of the sodium and potassium channel "gates"

## Challenges for next week:

6. Measure and plot an "f-I curve"
7. Prelude to model optimization 

## 1. Discuss solutions to two challenges from Lesson 1:

### Write a function to measure the "input resistance" of a neuron compartment

In [ ]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
from neuron import h
h.load_file('stdrun.hoc')

In [ ]:
def get_input_resistance(t, vm, i, start, stop, window_dur):
    
    baseline_start_index = np.where(t >= start - window_dur)[0][0]
    baseline_end_index = np.where(t >= start)[0][0]
    equil_start_index = np.where(t >= stop - window_dur)[0][0]
    equil_end_index = np.where(t >= stop)[0][0]
    delta_vm = np.abs(np.mean(vm[equil_start_index:equil_end_index]) - np.mean(vm[baseline_start_index:baseline_end_index]))
    delta_i = np.abs(np.mean(i[equil_start_index:equil_end_index]) - np.mean(i[baseline_start_index:baseline_end_index]))
    
    # Ohms = Volts / Amps; MegaOhms = milliVolts / nanoAmps
    input_res = delta_vm / delta_i
    return input_res

In [ ]:
soma = h.Section()
soma.L = 20.
soma.diam = 20.

soma.insert('pas')
leak_mechanism = soma(0.5).pas
pas_g0 = 0.0005
leak_mechanism.g = pas_g0  # conductance of leak channel in microSeimens
v_init = -65.
pas_e0 = v_init
leak_mechanism.e = pas_e0

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t)  # record the time stamp
soma_voltage.record(soma(0.5)._ref_v)  # record the voltage across the membrane in a segment

step_current_stim = h.IClamp(soma(0.5))
step_current_stim.amp = -0.1  # amplitude in nanoAmps
step_current_stim.dur = 200.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection

step_current_rec = h.Vector()
step_current_rec.record(step_current_stim._ref_i)

h.tstop = 600.

In [ ]:
h.run(v_init)

fig, axes = plt.subplots(1, 2, figsize=(8, 3))
axes[0].plot(t, step_current_rec, c='r')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Current (nA)')
axes[0].set_title('Injected current')
axes[1].plot(t, soma_voltage, c='k')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Voltage (mV)')
axes[1].set_title('Soma membrane potential')

input_resistance = get_input_resistance(np.array(t), np.array(soma_voltage), np.array(step_current_rec), start=200., stop=400., window_dur=10.)
print('The input resistance is %.2f MOhm' % input_resistance)

### Organize recordings and superimpose results from multiple simulations

In [ ]:
sim_history = []
stim_amp_array = np.arange(0., 0.6, 0.1)

for stim_amp in stim_amp_array:
    sim_record = {}
    sim_record['stim_amp'] = stim_amp
    step_current_stim.amp = stim_amp
    sim_record['description'] = '%.1f nA' % stim_amp
    h.run(v_init)
    sim_record['soma_voltage'] = np.array(soma_voltage)
    sim_record['t'] = np.array(t)
    sim_record['step_current_rec'] = np.array(step_current_rec)
    
    sim_history.append(sim_record)

In [ ]:
print(stim_amp_array)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8, 3))

for sim_record in sim_history:
    t_array = sim_record['t']
    soma_voltage_array = sim_record['soma_voltage']
    step_current_rec_array = sim_record['step_current_rec']
    description = sim_record['description']
    axes[0].plot(t_array, step_current_rec_array, label=description)
    axes[0].set_xlabel('Time (ms)')
    axes[0].set_ylabel('Current (nA)')
    axes[0].set_title('Injected current')
    axes[1].plot(t_array, soma_voltage_array)
    axes[1].set_xlabel('Time (ms)')
    axes[1].set_ylabel('Voltage (mV)')
    axes[1].set_title('Soma membrane potential')
axes[0].legend(loc='best', frameon=False)
fig.tight_layout(w_pad=0.2)

## 2. Introduction to sodium and potassium channels involved in generating action potentials (spikes)

<img src="images/hh_fig1.jpg" width="1000">

### Great resource:

https://ocw.mit.edu/courses/brain-and-cognitive-sciences/9-40-introduction-to-neural-computation-spring-2018/lecture-notes/MIT9_40S18_Lec04.pdf

## 3. Introduction to NMODL, the language used to specify ion channel mechanisms in NEURON

Earlier we inserted the ion channel mechanism 'pas' into our NEURON model to provide a passive "leak" current.

Now we want to insert sodium and potassium ion channels to allow our model to generate spikes.

Let's look at the code that describes the time- and voltage-dependent activation and inactivation of these channels:

Here are the contents of the built-in NEURON mechanism defined in 'hh.mod':

    TITLE hh.mod   squid sodium, potassium, and leak channels

    COMMENT
     This is the original Hodgkin-Huxley treatment for the set of sodium, 
      potassium, and leakage channels found in the squid giant axon membrane.
      ("A quantitative description of membrane current and its application 
      conduction and excitation in nerve" J.Physiol. (Lond.) 117:500-544 (1952).)
     Membrane voltage is in absolute mV and has been reversed in polarity
      from the original HH convention and shifted to reflect a resting potential
      of -65 mV.
     Remember to set celsius=6.3 (or whatever) in your HOC file.
     See squid.hoc for an example of a simulation using this model.
     SW Jaslove  6 March, 1992
    ENDCOMMENT

    UNITS {
            (mA) = (milliamp)
            (mV) = (millivolt)
        (S) = (siemens)
    }

    NEURON {
            SUFFIX hh
            USEION na READ ena WRITE ina
            USEION k READ ek WRITE ik
            NONSPECIFIC_CURRENT il
            RANGE gnabar, gkbar, gl, el, gna, gk
            GLOBAL minf, hinf, ninf, mtau, htau, ntau
    }

    PARAMETER {
            gnabar = .12 (S/cm2)	<0,1e9>
            gkbar = .036 (S/cm2)	<0,1e9>
            gl = .0003 (S/cm2)	<0,1e9>
            el = -54.3 (mV)
    }

    STATE {
            m h n
    }

    ASSIGNED {
            v (mV)
            celsius (degC)
            ena (mV)
            ek (mV)

        gna (S/cm2)
        gk (S/cm2)
            ina (mA/cm2)
            ik (mA/cm2)
            il (mA/cm2)
            minf hinf ninf
        mtau (ms) htau (ms) ntau (ms)
    }
    
    BREAKPOINT {
            SOLVE states METHOD cnexp
            gna = gnabar*m*m*m*h
        ina = gna*(v - ena)
            gk = gkbar*n*n*n*n
        ik = gk*(v - ek)      
            il = gl*(v - el)
    }


    INITIAL {
        rates(v)
        m = minf
        h = hinf
        n = ninf
    }
    
    DERIVATIVE states {  
            rates(v)
            m' =  (minf-m)/mtau
            h' = (hinf-h)/htau
            n' = (ninf-n)/ntau
    }

    :LOCAL q10

    
    PROCEDURE rates(v(mV)) {  :Computes rate and other constants at current v.
                          :Call once from HOC to initialize inf at resting v.
            LOCAL  alpha, beta, sum, q10
            TABLE minf, mtau, hinf, htau, ninf, ntau DEPEND celsius FROM -100 TO 100 WITH 200

    UNITSOFF
            q10 = 3^((celsius - 6.3)/10)
                    :"m" sodium activation system
            alpha = .1 * vtrap(-(v+40),10)
            beta =  4 * exp(-(v+65)/18)
            sum = alpha + beta
        mtau = 1/(q10*sum)
            minf = alpha/sum
                    :"h" sodium inactivation system
            alpha = .07 * exp(-(v+65)/20)
            beta = 1 / (exp(-(v+35)/10) + 1)
            sum = alpha + beta
        htau = 1/(q10*sum)
            hinf = alpha/sum
                    :"n" potassium activation system
            alpha = .01*vtrap(-(v+55),10) 
            beta = .125*exp(-(v+65)/80)
        sum = alpha + beta
            ntau = 1/(q10*sum)
            ninf = alpha/sum
    }

    FUNCTION vtrap(x,y) {  :Traps for 0 in denominator of rate eqns.
            if (fabs(x/y) < 1e-6) {
                    vtrap = y*(1 - x/y/2)
            }else{
                    vtrap = x/(exp(x/y) - 1)
            }
    }

    UNITSON

## 4. Inserting "Hodgkin-Huxley" ion channels into a NEURON model compartment

### First we need a default NEURON section, which mimics Hodgkin-Huxley's conditions:

In [ ]:
soma = h.Section()

soma.insert('hh')

t = h.Vector()
soma_voltage = h.Vector()
t.record(h._ref_t)  # record the time stamp
soma_voltage.record(soma(0.5)._ref_v)  # record the voltage across the membrane in a segment

step_current_stim = h.IClamp(soma(0.5))
step_current_stim.amp = 10.  # amplitude in nanoAmps
step_current_stim.dur = 200.  # duration in milliseconds
step_current_stim.delay = 200.  # start time of current injection

step_current_rec = h.Vector()
step_current_rec.record(step_current_stim._ref_i)

h.tstop = 600.

In [ ]:
soma.L, soma.diam

This mechanism specifies 3 distinct ion channels in one!
 - Passive leak conductance
 - Hodkin-Huxley-style sodium conductance with voltage-dependent activation gate and voltage-dependent inactivation gate
 - Hodkin-Huxley-style delayed rectifier potassium conductance with voltage-dependent activation gate

### Let's make sure it spikes when we inject current (and not when we do not).

In [ ]:
plt.figure()
for stim_amp in np.arange(0., 20., 5.):
    step_current_stim.amp = stim_amp
    h.run()
    plt.plot(t, soma_voltage, label='%.1f nA' % stim_amp)
plt.legend(loc='best', frameon=False)

### We can also record the actual currents through the sodium and potassium channels during a spike:

In [ ]:
soma_ina_rec = h.Vector()
soma_ina_rec.record(soma(0.5)._ref_ina)

soma_ik_rec = h.Vector()
soma_ik_rec.record(soma(0.5)._ref_ik)

step_current_stim.amp = 5.
step_current_stim.delay = 10.
step_current_stim.dur = 50.
h.tstop = 60.
h.run()

fig, axes = plt.subplots(3, figsize=(5, 10), sharex=True)
axes[0].plot(t, step_current_rec, c='r')
axes[0].set_xlabel('Time (ms)')
axes[0].set_ylabel('Current (nA)')
axes[0].set_title('Injected current')
axes[1].plot(t, soma_voltage, c='k')
axes[1].set_xlabel('Time (ms)')
axes[1].set_ylabel('Voltage (mV)')
axes[1].set_title('Soma membrane potential')
axes[2].plot(t, soma_ina_rec, label='I_Na')
axes[2].plot(t, soma_ik_rec, label='I_K')
axes[2].legend(loc='best', frameon=False)


## 5. Plotting the voltage-dependence of the sodium and potassium channel "gates"

From the hh.mod file:

gna = gnabar * m * m * m * h

gk = gkbar * n * n * n * n

gnabar = maximum conductance of the na channel

m = activation gate of the na channel

h = inactivation gate of the na channel

gkbar = maximum conductance of the k channel

n = activation gate of the k channel

In [ ]:
q10 = 1.  # For this purpose, temperature = 6.3 and the temperature sensitivity factor = 1.

def vtrap(x, y):
    if abs(x/y) < 1e-6:
        return y * (1. - x / y / 2.)
    else:
        return x / (np.exp(x / y) - 1.)

vtrap = np.vectorize(vtrap)
    
def mtau(v):
    alpha = .1 * vtrap(-(v+40), 10)
    beta =  4 * np.exp(-(v+65)/18)
    this_sum = alpha + beta
    return 1. / (q10 * this_sum)

def minf(v):
    alpha = .1 * vtrap(-(v+40), 10)
    beta =  4 * np.exp(-(v+65)/18)
    this_sum = alpha + beta
    return alpha / this_sum

def htau(v):
    alpha = .07 * np.exp(-(v+65)/20)
    beta = 1 / (np.exp(-(v+35)/10) + 1)
    this_sum = alpha + beta
    return 1 / (q10 * this_sum)

def hinf(v):
    alpha = .07 * np.exp(-(v+65)/20)
    beta = 1 / (np.exp(-(v+35)/10) + 1)
    this_sum = alpha + beta
    return alpha / this_sum

def ntau(v):
    alpha = .01*vtrap(-(v+55),10) 
    beta = .125*np.exp(-(v+65)/80)
    this_sum = alpha + beta
    return 1. / (q10 * this_sum)

def ninf(v):
    alpha = .01*vtrap(-(v+55),10) 
    beta = .125*np.exp(-(v+65)/80)
    this_sum = alpha + beta
    return alpha / this_sum

In [ ]:
v = np.linspace(-100., 100., 200)
fig, axes = plt.subplots(3, 2, figsize=(8, 8))
axes[0][0].plot(v, minf(v))
axes[0][0].set_xlabel('Voltage (mV)')
axes[0][0].set_ylabel('Activation')
axes[0][0].set_title('Na channel \'m\' activation gate')
axes[1][0].plot(v, hinf(v))
axes[1][0].set_xlabel('Voltage (mV)')
axes[1][0].set_ylabel('Inactivation')
axes[1][0].set_title('Na channel \'h\' inactivation gate')
axes[2][0].plot(v, ninf(v))
axes[2][0].set_xlabel('Voltage (mV)')
axes[2][0].set_ylabel('Activation')
axes[2][0].set_title('K channel \'n\' activation gate')

axes[0][1].plot(v, mtau(v))
axes[0][1].set_xlabel('Voltage (mV)')
axes[0][1].set_ylabel('Time constant (ms)')
axes[0][1].set_title('Na channel \'m\' activation kinetics')
axes[1][1].plot(v, htau(v))
axes[1][1].set_xlabel('Voltage (mV)')
axes[1][1].set_ylabel('Time constant (ms)')
axes[1][1].set_title('Na channel \'h\' inactivation kinetics')
axes[2][1].plot(v, ntau(v))
axes[2][1].set_xlabel('Voltage (mV)')
axes[2][1].set_ylabel('Time constant (ms)')
axes[2][1].set_title('K channel \'n\' activation kinetics')
fig.tight_layout()

# Challenges for next week:

## 6. Measure and plot an "f-I curve"

 - Run a series of simulations with different amplitudes of injected stimulus current
 - Compute the number of spikes - how do you find a spike from a voltage trace?
 - Plot the firing rate (f) (# of spikes / duration of time) versus input current (I)

## Prelude to model optimization:
 - What value of gl (leak conductance) sets the input resistance of the HH soma to 150 MOhm?
 - What values of gnabar and gkbar produce a spike at 0.1 nA instead of 5 nA?

# Some papers to read for next week:

 - Hoffman, Dax A., et al. "K+ channel regulation of signal propagation in dendrites of hippocampal pyramidal neurons." Nature 387.6636 (1997): 869-875. https://www.nature.com/articles/43119
 - Migliore, M., Hoffman, D. A., Magee, J. C. & Johnston, D. Role of an A-type K+ conductance in the back-propagation of action potentials in the dendrites of hippocampal pyramidal neurons. J Comput Neurosci 7, 5–15 (1999). https://link.springer.com/article/10.1023/A:1008906225285
 - Johnston, D. et al. Dendritic potassium channels in hippocampal pyramidal neurons. J Physiology 525 Pt 1, 75–81 (2000). https://physoc.onlinelibrary.wiley.com/doi/10.1111/j.1469-7793.2000.00075.x